# 1.什么是Word2Vec

Word2Vec是一种用于将词汇转换为向量表示的技术，它是自然语言处理中的一种重要工具。它的核心思想是将词汇映射到一个高维空间中的向量，使得具有相似含义的词汇在这个空间中的向量表示也相互靠近。这种表示能够捕捉到词汇之间的语义关系，例如近义词在向量空间中距离较近，而语义上不相关的词汇则距离较远。

Word2Vec模型主要有两种训练方法：Skip-gram和CBOW（Continuous Bag of Words）。Skip-gram模型通过给定中心词来预测上下文词汇，而CBOW则通过给定上下文词汇来预测中心词。这两种方法都能够学习到词汇的分布式表示，这些表示可以用作后续自然语言处理任务（如情感分析、命名实体识别等）的输入特征。

总之，Word2Vec通过将词汇转换为向量，有效地捕捉了词汇之间的语义信息，为自然语言处理提供了强大的工具。

#### **人话：**把文字转成向量，放到一个空间里，把近义词 尽量 放一块，把反义词/不相关的词 尽量 放远一些 ，有点像语义分类

# 2.怎么实现Word2Vec，有哪些步骤，这些步骤作用是什么

## 步骤1.数据预处理

作用：将原始文本数据转化为适合模型处理的格式。

## 步骤2. 构建词汇表

作用：创建一个将词汇映射到唯一索引的数据结构。

## 步骤3. 生成训练数据

作用：根据词汇表和上下文窗口创建训练样本。

## 步骤4. 定义模型

作用：建立神经网络模型以进行词向量学习。

## 步骤5. 训练模型

作用：优化模型参数以最小化预测误差。

## 步骤6. 评估模型

作用：检查模型的性能，并提取训练好的词向量。

## 步骤7. 应用模型

作用：将训练好的词向量应用于实际的自然语言处理任务。

# 3.代码实现

## 3.1 数据预处理
    1.去除文本中的标点符号和特殊字符，保留中文、英文和空格。
    2.将文本中的中文和英文分开处理，并将它们合并成一个单词列表。

### 3.1.1 数据预处理

#### **1.去除文本中的标点符号和特殊字符，保留中文、英文和空格。**

In [2]:
import re

def preprocess_text(text):
    # 去除所有标点符号和特殊字符（保留中英文和空格）
    text = re.sub(r'[^\w\s\u4e00-\u9fff]', '', text)
    return text

##### **示例**

In [3]:
# 示例文本
text = "我喜欢吃苹果, and I like bananas too!"
preprocessed_text = preprocess_text(text)
#去除了符号
print("预处理后的文本:", preprocessed_text)

预处理后的文本: 我喜欢吃苹果 and I like bananas too


#### **2.将文本中的中文和英文分开处理，并将它们合并成一个单词列表。**

In [4]:
import re

def tokenize_text(text):
    # 提取中文字符
    chinese_text = re.findall(r'[\u4e00-\u9fff]+', text)
    
    # 中文字符分词（每个字符作为一个词）
    chinese_tokens = []
    for chunk in chinese_text:
        for char in chunk:
            chinese_tokens.append(char)

    # 提取英文单词（包括连续的英文单词）
    english_tokens = re.findall(r'[a-zA-Z]+', text)
    
    # 合并所有分词结果
    all_tokens = chinese_tokens + english_tokens
    return all_tokens

##### **示例**

In [5]:
# 示例文本
text = "我喜欢吃苹果，and I like bananas too!"
tokens = tokenize_text(text)
print("分词后的文本数据:", tokens)

分词后的文本数据: ['我', '喜', '欢', '吃', '苹', '果', 'and', 'I', 'like', 'bananas', 'too']


#### **2.将文本中的中文和英文分开处理，并将它们合并成一个单词列表。（jieba版本）**

In [145]:
import re
import jieba

def tokenize_text_jieba(text):
    """
    使用 jieba 对中文进行分词，并对英文按空格分词。
    """
    # 提取中文字符
    chinese_text = re.findall(r'[\u4e00-\u9fff]+', text)
    
    # 中文字符分词
    chinese_tokens = []
    for chunk in chinese_text:
        chinese_tokens.extend(jieba.cut(chunk))
    
    
    # 提取英文单词（包括连续的英文单词）
    english_tokens = re.findall(r'[a-zA-Z]+', text)
    print(chinese_tokens)
    print(english_tokens)
    
    # 合并所有分词结果
    all_tokens = chinese_tokens + english_tokens
    return all_tokens


##### **示例**

In [146]:
# 示例文本
text = "我喜欢吃苹果 and I like bananas too!"
tokens = tokenize_text_jieba(text)
print("分词后的文本数据:", tokens)

['我', '喜欢', '吃', '苹果']
['and', 'I', 'like', 'bananas', 'too']
分词后的文本数据: ['我', '喜欢', '吃', '苹果', 'and', 'I', 'like', 'bananas', 'too']


## 3.2 构建词汇表
    1.统计每个词在所有文档中出现的频次
    2.根据词频统计构建词汇表

#### **1.统计每个词在所有文档中出现的频次。**

In [147]:
from collections import Counter

def count_word_frequencies(documents):
    all_words = []
    for doc in documents:
        for word in doc:
            all_words.append(word)
    #把所有词都放进数组后，使用Count方法进行统计
    word_counts = Counter(all_words)
    return word_counts

##### **示例**

In [148]:
# 示例文档
documents = [
    ['我', '喜欢', '吃', '苹果'],
    ['我', '喜欢', '吃', '香蕉'],
    ['苹果', '和', '香蕉', '都是', '水果'],
    ['汽车', '很', '方便'],
    ['我', '有', '一只', '狗']
]

# 计算词频
word_counts = count_word_frequencies(documents)
print("词频统计:", word_counts)

词频统计: Counter({'我': 3, '喜欢': 2, '吃': 2, '苹果': 2, '香蕉': 2, '和': 1, '都是': 1, '水果': 1, '汽车': 1, '很': 1, '方便': 1, '有': 1, '一只': 1, '狗': 1})


#### **2. 根据词频统计构建词汇表。**

In [149]:
#词出现的越频繁，词在词汇表越前面
def build_vocab(word_counts):
    vocab = {}
    print(word_counts.items())
    for idx, (word, _) in enumerate(word_counts.items()):
        #vocab[我]=当前序号
        vocab[word] = idx
    return vocab

##### **示例**

In [150]:
# 示例文档
documents = [
    ['我', '喜欢', '吃', '苹果'],
    ['我', '喜欢', '吃', '香蕉'],
    ['苹果', '和', '香蕉', '都是', '水果'],
    ['汽车', '很', '方便'],
    ['我', '有', '一只', '狗']
]

# 计算词频
word_counts = count_word_frequencies(documents)
print("词频统计:", word_counts)

# 构建词汇表
vocab = build_vocab(word_counts)
print("词汇表:", vocab)

词频统计: Counter({'我': 3, '喜欢': 2, '吃': 2, '苹果': 2, '香蕉': 2, '和': 1, '都是': 1, '水果': 1, '汽车': 1, '很': 1, '方便': 1, '有': 1, '一只': 1, '狗': 1})
dict_items([('我', 3), ('喜欢', 2), ('吃', 2), ('苹果', 2), ('香蕉', 2), ('和', 1), ('都是', 1), ('水果', 1), ('汽车', 1), ('很', 1), ('方便', 1), ('有', 1), ('一只', 1), ('狗', 1)])
词汇表: {'我': 0, '喜欢': 1, '吃': 2, '苹果': 3, '香蕉': 4, '和': 5, '都是': 6, '水果': 7, '汽车': 8, '很': 9, '方便': 10, '有': 11, '一只': 12, '狗': 13}


## 3.3 生成训练数据

#### **定义 Word2VecDataset 类。**

In [151]:
from torch.utils.data import Dataset, DataLoader
import torch

class Word2VecDataset(Dataset):
    def __init__(self, documents, vocab, window_size=2):
        """
        初始化数据集，生成上下文-中心词对。

        参数:
        - documents: 一个包含分词后的文档列表，每个文档是一个词的列表。
        - vocab: 一个词汇表，将每个词映射到唯一的索引。
        - window_size: 上下文窗口的大小，默认为 2。
        """
        self.window_size = window_size  # 上下文窗口的大小
        self.vocab = vocab  # 词汇表
        self.pairs = self._create_pairs(documents)  # 生成上下文-中心词对
    
    def _create_pairs(self, documents):
        """
        创建上下文-中心词对。

        参数:
        - documents: 一个包含分词后的文档列表，每个文档是一个词的列表。

        返回:
        - pairs: 一个包含上下文-中心词对的列表。
        """
        pairs = []  # 用于存储上下文-中心词对的列表
        for doc in documents:
            for idx, word in enumerate(doc):
                word_idx = self.vocab[word]  # 获取中心词的索引
                # 定义上下文窗口的起始和结束位置
                #开始的索引 从0与 当前索引-窗口大小值 中选，也就是说如果减出来的值小于0 就为0
                start = max(0, idx - self.window_size)
                #结束的索引 从 数据集的长度和 当前索引窗口+窗口大小+1 决定
                ## +1 是因为 range 函数的 end 是不包括的
                end = min(len(doc), idx + self.window_size + 1)
                # 遍历上下文窗口中的所有词
                for context_idx in range(start, end):
                    if context_idx != idx:  # 跳过中心词
                        #当前中心词范围的值
                        context_word = doc[context_idx]
                        #当前中心词范围值的索引
                        context_word_idx = self.vocab[context_word]  # 获取上下文词的索引
                        pairs.append((word_idx, context_word_idx))  # 添加上下文-中心词对到列表
                        # 上下文词索引和中心词对的生成：
                        
                        # context_idx = 0，上下文词为 '我'，索引为 0，对 (2, 0)。
                        # context_idx = 1，上下文词为 '喜欢'，索引为 1，对 (2, 1)。
                        # context_idx = 3，上下文词为 '苹果'，索引为 3，对 (2, 3)。
                        # 最终生成的 pairs 列表包含：[(2, 0), (2, 1), (2, 3)]。
                        
        return pairs

    # pairs 列表生成的例子
# pairs = [
#     (0, 1), (1, 0), (1, 2), (2, 1), (2, 3), (3, 2),  # 文档 1: ['我', '喜欢', '吃', '苹果']
#     (0, 1), (1, 0), (1, 2), (2, 1), (2, 4), (4, 2),  # 文档 2: ['我', '喜欢', '吃', '香蕉']
#     (3, 5), (3, 4), (4, 3), (4, 5), (4, 6), (6, 4), (6, 7), (7, 6),  # 文档 3: ['苹果', '和', '香蕉', '都是', '水果']
#     (8, 9), (9, 8), (9, 10), (10, 9),  # 文档 4: ['汽车', '很', '方便']
#     (0, 11), (11, 0), (11, 12), (12, 11), (12, 13), (13, 12)  # 文档 5: ['我', '有', '一只', '狗']
# ]

    def __len__(self):
        """
        返回数据集中样本的总数。
        """
        return len(self.pairs)

    def __getitem__(self, idx):
        """
        返回指定索引的样本。

        参数:
        - idx: 样本的索引。

        返回:
        - 一个包含中心词和上下文词索引的元组。
        """
        center, context = self.pairs[idx]
        #print("中心词索引:", center)  # tensor(0)
        #print("上下文词索引:", context)  # tensor(1)
        return torch.tensor(center, dtype=torch.long), torch.tensor(context, dtype=torch.long)


##### **示例**

In [220]:
from torch.utils.data import random_split

# 创建数据集
dataset = Word2VecDataset(documents, vocab)
#因为被打乱了所以是乱序输出
#batch_size=2 一次输出2个
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 数据集划分为训练集和验证集
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 创建数据加载器
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# 打印示例
for center, context in dataloader:
    print("中心词索引:", center)
    print("上下文词索引:", context)

中心词索引: tensor([5, 4])
上下文词索引: tensor([4, 2])
中心词索引: tensor([12,  3])
上下文词索引: tensor([11,  1])
中心词索引: tensor([5, 0])
上下文词索引: tensor([3, 2])
中心词索引: tensor([6, 9])
上下文词索引: tensor([4, 8])
中心词索引: tensor([11,  7])
上下文词索引: tensor([12,  6])
中心词索引: tensor([ 0, 13])
上下文词索引: tensor([12, 12])
中心词索引: tensor([ 2, 12])
上下文词索引: tensor([4, 0])
中心词索引: tensor([4, 1])
上下文词索引: tensor([5, 2])
中心词索引: tensor([8, 4])
上下文词索引: tensor([10,  3])
中心词索引: tensor([2, 0])
上下文词索引: tensor([3, 1])
中心词索引: tensor([ 2, 10])
上下文词索引: tensor([1, 9])
中心词索引: tensor([1, 3])
上下文词索引: tensor([0, 5])
中心词索引: tensor([9, 2])
上下文词索引: tensor([10,  0])
中心词索引: tensor([6, 4])
上下文词索引: tensor([7, 1])
中心词索引: tensor([7, 1])
上下文词索引: tensor([4, 0])
中心词索引: tensor([11,  1])
上下文词索引: tensor([0, 2])
中心词索引: tensor([4, 1])
上下文词索引: tensor([7, 3])
中心词索引: tensor([8, 2])
上下文词索引: tensor([9, 1])
中心词索引: tensor([4, 0])
上下文词索引: tensor([6, 1])
中心词索引: tensor([ 3, 10])
上下文词索引: tensor([4, 8])
中心词索引: tensor([13,  0])
上下文词索引: tensor([11,  2])
中心词索引: tensor([ 3, 12])
上下文

## 3.4 定义模型

#### **基础模型：embedding。**

In [153]:
import torch

class MyEmbedding:
    def __init__(self, vocab_size, embedding_dim):
        """
        初始化词嵌入矩阵。

        参数:
        - vocab_size: 词汇表的大小
        - embedding_dim: 嵌入向量的维度
        """
        # 初始化嵌入矩阵为随机值
        self.embeddings = torch.randn(vocab_size, embedding_dim, requires_grad=True)

    def forward(self, indices):
        """
        根据词索引获取嵌入向量。

        参数:

        indices：类似于学生的学号（或索引），你用它来查找具体的学生档案。
        self.embeddings[indices]：根据学号（或索引），从档案表格中提取出相应的档案（嵌入向量）。
        返回:
        - 对应的嵌入向量（张量）
        """
        return self.embeddings[indices]

    def __getitem__(self, indices):
        """
        支持索引操作以获取嵌入向量。

        参数:
        - indices: 词的索引（可以是一个列表或张量）

        返回:
        - 对应的嵌入向量（张量）
        """
        return self.forward(indices)

In [154]:
# 示例：
# 假设你有一个班级有 10 名学生，每个学生的档案包含 5 个特征（例如成绩、身高、体重等）。你用学生的学号来从档案表格中提取学生的档案信息。

#
# 在这个例子中，词嵌入就是班级的学生档案。
# 你为每个学生创建一个唯一的档案（词向量），并能通过学号（索引）快速查找每个学生的档案。
# MyEmbedding 类通过这种方式存储和检索每个词的嵌入向量。

# 示例
vocab_size = 10  # 词汇表大小
embedding_dim = 5  # 嵌入向量的维度

# 创建词嵌入对象
embedding = MyEmbedding(vocab_size, embedding_dim)

# 示例词索引
indices = torch.tensor([0, 1, 2, 3])

# 获取词嵌入
embeds = embedding.forward(indices)

print("词嵌入矩阵:\n", embedding.embeddings)
print("获取的词嵌入:\n", embeds)

词嵌入矩阵:
 tensor([[-1.3036,  0.2762,  1.2268,  1.4505,  0.0286],
        [ 0.4403,  0.2579, -0.1095, -0.5406, -0.4847],
        [ 2.7089, -0.1263, -0.6482,  1.1521, -1.1351],
        [ 0.2351, -1.2752,  1.5673,  1.0044,  1.0660],
        [-0.2077,  1.2500,  0.6497,  1.1076, -0.5300],
        [ 1.9304, -0.7198, -1.7162,  0.5596,  1.7736],
        [-0.6066, -0.2974,  0.0458, -1.6658,  1.6694],
        [-0.7357, -0.1079, -0.1545, -0.4720,  0.7539],
        [ 0.1120, -0.7730,  0.7116, -0.3670,  0.1294],
        [ 1.8195,  0.0517,  2.6299,  0.1626,  1.7854]], requires_grad=True)
获取的词嵌入:
 tensor([[-1.3036,  0.2762,  1.2268,  1.4505,  0.0286],
        [ 0.4403,  0.2579, -0.1095, -0.5406, -0.4847],
        [ 2.7089, -0.1263, -0.6482,  1.1521, -1.1351],
        [ 0.2351, -1.2752,  1.5673,  1.0044,  1.0660]],
       grad_fn=<IndexBackward0>)


#### **Word2Vec模型：Skip-Gram。**

In [172]:
import torch
import torch.nn as nn

class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGram, self).__init__()
        # 学生兴趣档案（输入词嵌入矩阵）
        self.in_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 课程特征档案（输出词嵌入矩阵）
        self.out_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 损失函数
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, center):
        in_embeds = self.in_embeddings(center)
        return in_embeds
        
    def loss(self, scores, target):
        return self.loss_fn(scores, target)  # target 应该是 (batch_size,)

## 3.5 训练模型

#### **拆分后的代码**

In [225]:
import torch.optim as optim

# 初始化模型
vocab_size = len(vocab)
embedding_dim = 100  # 嵌入维度
model = SkipGram(vocab_size, embedding_dim)

# 选择优化器
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [234]:

# 训练超参数
num_epochs = 5

for epoch in range(num_epochs):
    total_loss = 0
    #放训练集训练
    for center, context in train_dataloader:
        optimizer.zero_grad()
        
        # 前向传播
        # 通过模型的前向传播方法，获取当前批次中每个中心词的嵌入向量
        center_embeds = model(center)  # (batch_size, embedding_dim)

        # 获取模型中所有上下文词的嵌入矩阵，形状为 (vocab_size, embedding_dim)
        # 这里的上下文词嵌入矩阵是通过模型的输出嵌入层的权重获得的
        context_embeds = model.out_embeddings.weight  # (vocab_size, embedding_dim)
        
        # 计算中心词嵌入与所有上下文词嵌入之间的点积
        # center_embeds 形状为 (batch_size, embedding_dim)
        # context_embeds.t() 形状为 (embedding_dim, vocab_size)，是上下文词嵌入矩阵的转置
        # 结果 scores 形状为 (batch_size, vocab_size)，表示每个中心词对所有可能上下文词的得分
        scores = torch.matmul(center_embeds, context_embeds.t())
        
        # CrossEntropyLoss 需要 target 是正确的类别索引
        loss = model.loss(scores, context)
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader)}")

Epoch 1/5, Loss: 1.1250261294841766
Epoch 2/5, Loss: 1.1370073390007018
Epoch 3/5, Loss: 1.167758914232254
Epoch 4/5, Loss: 1.126306984424591
Epoch 5/5, Loss: 1.1364572167396545


#### **完整代码**

In [262]:
import torch.optim as optim

# 初始化模型
vocab_size = len(vocab)
embedding_dim = 100  # 嵌入维度
model = SkipGram(vocab_size, embedding_dim)

# 选择优化器
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练超参数
num_epochs = 5

for epoch in range(num_epochs):
    total_loss = 0
    for center, context in train_dataloader:
        optimizer.zero_grad()
        
        # 前向传播
        # 通过模型的前向传播方法，获取当前批次中每个中心词的嵌入向量
        center_embeds = model(center)  # (batch_size, embedding_dim)

        # 获取模型中所有上下文词的嵌入矩阵，形状为 (vocab_size, embedding_dim)
        # 这里的上下文词嵌入矩阵是通过模型的输出嵌入层的权重获得的
        context_embeds = model.out_embeddings.weight  # (vocab_size, embedding_dim)
        
        # 计算中心词嵌入与所有上下文词嵌入之间的点积
        # center_embeds 形状为 (batch_size, embedding_dim)
        # context_embeds.t() 形状为 (embedding_dim, vocab_size)，是上下文词嵌入矩阵的转置
        # 结果 scores 形状为 (batch_size, vocab_size)，表示每个中心词对所有可能上下文词的得分
        scores = torch.matmul(center_embeds, context_embeds.t())
        
        # CrossEntropyLoss 需要 target 是正确的类别索引
        loss = model.loss(scores, context)
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader)}")

Epoch 1/5, Loss: 11.065189571380616
Epoch 2/5, Loss: 9.265730533599854
Epoch 3/5, Loss: 7.8241242408752445
Epoch 4/5, Loss: 6.782663764953614
Epoch 5/5, Loss: 5.937912817001343


## 3.6 评估模型

#### **评估模型**

In [238]:
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for center, context in dataloader:
            center_embeds = model(center)
            context_embeds = model.out_embeddings.weight
            scores = torch.matmul(center_embeds, context_embeds.t())
            loss = model.loss(scores, context)
            total_loss += loss.item()
    return total_loss / len(dataloader)

In [263]:
avg_loss = total_loss / len(train_dataloader)
val_loss = evaluate(model, val_dataloader)  # 在验证集上评估损失
print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}, Val Loss: {val_loss}")

Epoch 5/5, Loss: 7.422391021251679, Val Loss: 14.03045883178711


#### **保存模型:**

In [199]:
torch.save(model.state_dict(), 'skipgram_model.pth')
print("训练和评估完成，模型已保存！")

训练和评估完成，模型已保存！


#### **可视化词嵌入(TODO)**